<a href="https://colab.research.google.com/github/juanlu29/juanlu29/blob/gp_aprendizaje/libreriaGP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Modulos y constantes

import numpy as np
import random
import scipy
import scipy.linalg
from matplotlib import pyplot as plt
import seaborn as sns
from google.colab import drive
import pandas as pd
import numba

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# Pruebas con funciones anonimas en python

class funcionesPrueba():
  def __init__(self):
    '''
    Covarianza puede ser una cadena de texto o una funcion anónima como combinación de metodos de la clase
    covarianza es un diccionario
    '''
    self.Kernels = self.kernels()

  def calcCovM(self,x,y):
    '''
    Dados x e y como ndarrays, calcula la matriz de covarianzas cuyos elementos son evaluados de la forma
    C(xi,yi) = (k(xi,yi)) con rango (nx x ny) donde nx y ny son la longitud de los vectores x  e y respectivamente.
    hiper son los hiperparámetros usados en el modelo
    '''
    # Se crean matrices [[x],[x],...] (longitud de la concatenacion nx) y [[y],[y],...].T (longitud de la concatenacion ny)
    xM = np.full((len(y),len(x)),x)
    yM = (np.full((len(x),len(y)),y)).T
    # Llamamos la funcion Ker que nos debe devolver como salida una matriz cuyos elementos calculados representan k(x,y)
    return self.Ker(xM,yM,*self.Hiper)


  def compCov(self,covarianza,*hiper):
    '''
    Permite definir el kernel a usar por el proceso gaussiano, tanto si es uno de los kernels definidos en la clase como si es una combinacion de los mismos
    El argumento es una funcion anónima en caso de ser una composición, en caso contrario es suficiente con proporcionar el nombre del kernel definido
    hiper son los hiperparámetros que precisan el kernel y vienen dado por un diccionario
    '''
    self.Hiper = hiper
    self.Ker = lambda x_test, x_obs, *hiper: covarianza(x_test,x_obs,*self.Hiper) # Covarianza definida

  class kernels():
    '''
    Se definen kernels de uso común junto a sus hiperparámetros
    '''
    def __init__(self):
      return

    def exponencialCuadrada(self,xb,xa,s,l):
      '''
      Correlacion estacionaria (depende solo de la distancia r = abs(xb-xa)) con decaimiento exponencial cuadratico
      Hiperparámetros:
      s : amplitud de la correlacion, esta relacionado con la amplitud de las trayectorias generadas por el GP
      l : escala de longitud del GP. Esta relacionada con la distancia sobr la cual los procesos generados por el GP pueden presentar oscilaciones o variaciones
      '''
      return np.power(s,2)*np.exp(-np.power(xa-xb,2)/(2.*np.power(l,2)))

    def ruidoBlanco(self,xb,xa,sigma):
      '''
      Correlaciones de ruido blanco o gaussiano puro, solo correlaciona los puntos consigo mismo. 
      Representa incertidumbres intrínsecas en las observaciones y asume una distribución a priori de esos errores gaussiano
      Hiperparáámetros:
      s : amplitud o incertidumbre del ruido generado
      '''
      return np.where(xb==xa, np.power(sigma,2),0.) # Se usa el metodo numpy.where porque las operaciones logicas sobre arrays son consideradas ambiguas en el intérprete de python
                                                    # Esta manera es válidad para python para comprar elemento a elemento entre matrices.

In [ ]:
x_test = np.linspace(1.,5.,4)
x_obs = np.linspace(1.,5.,4)
x_obs = x_obs.T
print(x_test,x_obs)
prueba = funcionesPrueba()
s = 3.
l = 4.
sigma = 0.
prueba.compCov(lambda x,y,s,l,sigma: prueba.Kernels.exponencialCuadrada(x,y,s,l)+prueba.Kernels.ruidoBlanco(x,y,sigma),s,l,sigma) # Aqui especificamos los hiperparametros
print(prueba.calcCovM(x_test,x_obs,s,l,sigma))

[1.         2.33333333 3.66666667 5.        ] [1.         2.33333333 3.66666667 5.        ]
[[9.         8.51363522 7.20663663 5.45877594]
 [8.51363522 9.         8.51363522 7.20663663]
 [7.20663663 8.51363522 9.         8.51363522]
 [5.45877594 7.20663663 8.51363522 9.        ]]


In [ ]:
print(x_test,x_obs)
x = 3.66666667
y = 5.
a = 3.*3.*np.exp(-np.power((x-y),2)/(2.*4.*4.))
print(a)

[1.         2.33333333 3.66666667 5.        ] [1.         2.33333333 3.66666667 5.        ]
8.513635222525789


In [60]:
class gaussProcess():
  def __init__(self,dominio,observacion,*hiper):
    '''
    Los objetos de esta clase modelan procesos gaussianos caracterizados por su promedio y covarianza
    Como argumento inicial incluimos el data set sobre el que se va a trabajar, su dominio y las observaciones como ndarrays
    '''
    #self.Derivadas = prior_Partial_Der
    self.Kernels = self.kernels()
    self.Dominio = dominio
    self.Observacion = observacion
    self.Hiper = hiper
    # Se crean matrices [[x],[x],...] (longitud de la concatenacion nx) y [[y],[y],...].T (longitud de la concatenacion ny)
    # Son necesarios para devolver los resultados necesarios en los calculos matriciales
    self.xM = np.full((len(self.Observacion),len(self.Dominio)),self.Dominio)
    self.yM = (np.full((len(self.Dominio),len(self.Observacion)),self.Observacion))

  def calcCovM(self,xM,yM):
    '''
    Dados x e y como ndarrays, calcula la matriz de covarianzas cuyos elementos son evaluados de la forma
    C(xi,yi) = (k(xi,yi)) con rango (nx x ny) donde nx y ny son la longitud de los vectores x  e y respectivamente.
    hiper son los hiperparámetros usados en el modelo
    '''
    # Llamamos la funcion Ker que nos debe devolver como salida una matriz cuyos elementos calculados representan k(x,y)
    return self.Ker(xM,yM.T,*self.Hiper)

  def calcDerCovM(self,xM,yM):
    '''
    Dados x e y como ndarrays, calcula la matriz correspondiente a la derivada respecto los hiperparámetros de las covarianzas
    cuyos elementos son evaluados de la forma C_j(xi,yi) = (dk(xi,yi)dh_j) con rango (nx x ny) donde nx y ny son la longitud de los 
    vectores x e y respectivamente. Hiper son los hiperparámetros usados en el modelo
    '''
    # Generamos el array de matrices llamando al zip de las funciones de derivadas y sus hiperparámetros para generarlas
    return np.asarray([ self.DerKer[i](xM,yM.T,*self.Hiper) for i in range(len(self.DerKer)) ] )

  def compCov(self,covarianza):
    '''
    Permite definir el kernel a usar por el proceso gaussiano, tanto si es uno de los kernels definidos en la clase como si es una combinacion de los mismos
    El argumento es una funcion anónima en caso de ser una composición, en caso contrario es suficiente con proporcionar el nombre del kernel definido
    hiper son los hiperparámetros que precisan el kernel y vienen dado por un diccionario
    '''
    self.Ker = covarianza # Covarianza definida
    self.Ker = np.vectorize(self.Ker)

  def compDerCov(self,derCovarianza):
    '''
    Permite definir la expresion para la derivada del kernel respecto los distintos hiperparametros como un iterable zip
    de funciones anonimas juntos a sus hiperparámetros. 
    En este caso por propositos de optimizacion se espera que se evalue dinamicamente con distintos hiperparámetros.
    Los datos contenidos en *hiper están encapsulados en una tupla a la que se debe acceder previamente
    '''
    try:
      self.DerKer = derCovarianza
    except:
      raise NameError("No se ha definido kernel previamente")


  class kernels():
    '''
    Se definen kernels de uso común junto a sus hiperparámetros y derivadas asociadas
    '''
    def __init__(self):
      return

    def exponencialCuadrada(self,xb,xa,s,l):
      '''
      Correlacion estacionaria (depende solo de la distancia r = abs(xb-xa)) con decaimiento exponencial cuadratico

      k(xb,xa) = s² exp( - (xb-xa)²/(2 l²) )

      Hiperparámetros:
      s : amplitud de la correlacion, esta relacionado con la amplitud de las trayectorias generadas por el GP
      l : escala de longitud del GP. Esta relacionada con la distancia sobr la cual los procesos generados por el GP pueden presentar oscilaciones o variaciones
      '''
      return np.power(s,2)*np.exp(-np.power(xa-xb,2)/(2.*np.power(l,2)))

    def expCua_der_s(self,xb,xa,s,l):
      '''
      Derivada respecto el hiperparámetro s de las correlaciones estacionarias con decaimiento exponenciales cuadráticas

      k(xb,xa) = 2s exp( - (xb-xa)²/(2 l²) )

      Hiperparámetros:
      s : amplitud de la correlacion, esta relacionado con la amplitud de las trayectorias generadas por el GP
      l : escala de longitud del GP. Esta relacionada con la distancia sobr la cual los procesos generados por el GP pueden presentar oscilaciones o variaciones
      '''
      return 2*s*np.exp(-np.power(xa-xb,2)/(2.*np.power(l,2)))

    def expCua_der_l(self,xb,xa,s,l):
      '''
      Derivada respecto el hiperparámetro l de las correlaciones estacionarias con decaimiento exponenciales cuadráticas

      k(xb,xa) = s²(xb-xa)² / l³ exp( - (xb-xa)²/(2 l²) )

      Hiperparámetros:
      s : amplitud de la correlacion, esta relacionado con la amplitud de las trayectorias generadas por el GP
      l : escala de longitud del GP. Esta relacionada con la distancia sobr la cual los procesos generados por el GP pueden presentar oscilaciones o variaciones
      '''
      return (np.power(s*(xa-xb),2)/np.power(l,3))*np.exp(-np.power(xa-xb,2)/(2.*np.power(l,2)))

    def ruidoBlanco(self,xb,xa,sigma):
      '''
      Correlaciones de ruido blanco o gaussiano puro, solo correlaciona los puntos consigo mismo. 

      k(xb,xa) = sigma² delta(xb,xa)

      Representa incertidumbres intrínsecas en las observaciones y asume una distribución a priori de esos errores gaussiano
      Hiperparáámetros:
      s : amplitud o incertidumbre del ruido generado
      '''
      return np.where(xb==xa, np.power(sigma,2),0.) # Se usa el metodo numpy.where porque las operaciones logicas sobre arrays son consideradas ambiguas en el intérprete de python
                                                    # Esta manera es válidad para python para comprar elemento a elemento entre matrices.

    def rB_der_sigma(self,xb,xa,sigma):
      '''
      Derivada respecto el hiperparámetro sigma del ruido no correlacionado

      k(xb,xa) = sigma² delta(xb,xa)

      Representa incertidumbres intrínsecas en las observaciones y asume una distribución a priori de esos errores gaussiano
      Hiperparáámetros:
      s : amplitud o incertidumbre del ruido generado
      '''
      return np.where(xb==xa, 2*sigma,0.) # Se usa el metodo numpy.where porque las operaciones logicas sobre arrays son consideradas ambiguas en el intérprete de python
                                          # Esta manera es válidad para python para comprar elemento a elemento entre matrices.

 
  def cholDescomp(self,K):
    '''
    Descomposicion cholesky de una matriz dada
    '''
    try:
      L = scipy.linalg.cholesky(K, lower=True)
    except:
     # L = scipy.linalg.cholesky(K + np.diag(0.000001*np.ones(int(np.sqrt(K.size)))), lower=True)
      raise NameError("No es posible efectuar descomposicion cholesky")

    return L

  def calcInvK(self,matriz):
    '''
    Calculo de la inversa de matriz. 
    '''
    # Calculo de matriz inversa de la covarianza. 
    try:
      invK = np.linalg.inv(matriz)
    except:
      raise NameError("No es posible calcular matriz inversa")

    return invK

  def condicionarGP(self,*dominio_test):
    '''
    Dadas unas observaciones junto a sus valores de dominio (uni-dimensionales) en el rango sobre el que se modela el GP, 
    obtiene la covarianza asociada a la distribución de trayectorias condicionadas al conjunto de partida dado un dominio 
    de test en el que queremos comprobar la predicción. Por tanto permite generar las trayectorias compatibles con los datos

    También calcula el logaritmo de la probabilidad de verosimilitud marginal (marginal likelihood) de las observaciones dados los inputs 
    y parámetros del modelo usado.

    Este algoritmo esta especificado en el libro "Gaussian Processes for Machine Learning", como algoritmo 2.1
    '''
    self.xtest = np.reshape(dominio_test,(len(dominio_test),1))

    # Definimos nueva covarianza
    # Primero se realiza la descomposición cholesky de la adición de la covarianza de la distribución a priori,
    # para lo cual calculamos la matriz de covarianza asociada al dominio de las observaciones sobre si mismo
    # y al cruce del dominio de las observaciones y del test de prediccion.
    try:
      self.Cov_obs_obs = self.calcCovM(self.xM,self.xM,*self.Hiper)
      self.Cov_obs_test = self.calcCovM(self.xM,self.xtest,*self.Hiper)
    except:
      raise NameError("Debes especificar primero el kernel que estas usando junto a sus hiperparametros con el metodo self.compCov")

    cholL = np.linalg.cholesky(self.Cov_obs_obs)

    # Calculo del vector alfa como solucion del sistema K_obs_obs*alfa = y_entrenamiento usando la descomposición cholesky anterior
    self.Alfa = scipy.linalg.cho_solve((cholL,True),self.yM)

    # Vector v solución particular del sistema cholL*v = k_test para todos los datos del entrenamiento.
    # Este sistema se resuelve introduciendo k_star no como matriz columna sino como la submatriz calculada como la evaluación de la covarianza a priori de los inputs de los valores a predecir y los inputs del entrenamiento
    v = scipy.linalg.solve_triangular(cholL,self.cov_obs_test, lower=True)

    # Se obtiene el valor promedio del proceso predictivo condicionado a las observaciones
    self.Media_pred = np.matmul(self.Cov_obs_test.T,self.Alfa)

    # Covarianza predictiva o condicionada. El proceso llevado a cabo ha sido obtener la distribución condicionada del GP a las observaciones
    # Intenta calcular directamente la matriz, en caso contrario calcular la matriz de covarianza de la distribucion a priori dado el test
    try:
      self.Cov_pred = self.Cov_priori - np.matmul(v.T,v)
    except:
      # En caso que no esté definida o no coincida con la dimensionalidad del test, se recalcula la covarianza a priori
      self.Cov_priori = self.calcCovM(self.xtest,self.xtest,*self.Hiper)
      self.Cov_pred = self.Cov_priori - np.matmul(v.T,v)

    # La siguiente matriz descompuesta es necesaria para generar procesos del GP.
    self.L_pred = self.cholDescomp(self.Cov_pred)
  

  def log_prob_verosimilitud_datos(self,*hiper):
    '''
    Dadas unas observaciones, kernel e hiperparámetros dados, calcula el logaritmo de la probabilidad de verosimilitud de las observaciones al modelo y sus parámetros
    '''

    # Actualizamos hiperparámetros
    self.Hiper = hiper

    # Primero se realiza la descomposición cholesky de la adición de la covarianza de la distribución a priorir y el término de ruido.
    #cholL = np.linalg.cholesky(self.calcCovM(dominio,dominio,*self.Hiper))
    cholL = np.linalg.cholesky(self.calcCovM(self.xM,self.xM))

    # Calculo del vector alfa como solucion del sistema K*alfa = y_entrenamiento usando la descomposición cholesky anterior
    alfa = scipy.linalg.cho_solve((cholL,True),self.Observacion)

    # Calculo de la probabilidad de verosimilitud marginal a este modelo y sus parámetros
    self.log_marg_y = - 0.5*np.dot(alfa,self.Observacion) - np.sum(np.log(np.diag(cholL)),axis=0) - (float(len(self.Dominio))/2.)*np.log(2*np.pi)
    if (self.log_marg_y < -70.):
      self.log_marg_y = 0.
    return -self.log_marg_y

  def der_log_prob_ver_hiper(self,*hiper):
    '''
    Derivada del logaritmo de la probabilidad de verosimilitud respecto hiperapametros,
    devuelve un ndarray con cada componenete correspondiendo a la matriz del modelo respecto
    los distintos. La expresion es la siguiente

    1/2 traza[  {alfa*alfa.T - K⁻¹} dK/dh] ; alfa = K⁻¹\y

    '''

    # Actualizamos hiperparámetros
    self.Hiper = hiper

    # Definimos nueva covarianza
    try:
      covDom = self.calcCovM(self.xM,self.xM)
    except:
      raise NameError("Debes especificar primero el kernel que estas usando junto a sus hiperparametros con el metodo self.compCov")


    invCovDom = self.calcInvK(covDom)

    cholL = np.linalg.cholesky(covDom)

    # Calculo del vector alfa como solucion del sistema K_obs_obs*alfa = y_entrenamiento usando la descomposición cholesky anterior
    alfa = scipy.linalg.cho_solve((cholL,True),self.Observacion)

    alfa = np.reshape(alfa,(len(alfa),1))

    self.Der_Hiper = hiper
    derCovHiper = self.calcDerCovM(self.xM,self.xM)
    return -0.5*np.trace(np.matmul(np.matmul(alfa,alfa.T)-invCovDom,derCovHiper),axis1=1,axis2=2) # La suma de la traza en un array de mas de 2 dimensiones debe especificarse sobre sus indices. En este caso es dimensióón 3.
    

  def prioriGP(self,*dominio):
    '''
    Computa los términos necesarios para extraer realizaciones del GP siguiendo la distribución a priori
    '''

    self.xtest = np.reshape(dominio_test,(len(dominio_test),1))

    try:
      self.Cov_priori = self.calcCovM(self.xtest,self.xtest,*self.Hiper)
    except:
      raise NameError("Debes especificar primero el kernel que estas usando junto a sus hiperparametros con el metodo self.compCov")

    # La siguiente matriz descompuesta es necesaria para generar procesos del GP.
    self.L_priori = self.cholDescomp(self.Cov_pred)


  def procesosPrioriGP(self):
    '''
    Genera realizaciones del proceso gaussiano dada la descomposicion cholesky de la matriz de covarianzas y la media correspondiente
    '''
    # Si no se ha calculado la descomposicióón de la covarianza se llama al método aqui
    try:
      return self.Mean_priori +  np.matmul(self.L_priori,(np.randn(len(self.xtest))).T)
    except:
      raise NameError("No se ha incluido los valores del dominio sobre los que muestrear el proceso o no se ha calculado la descomposición de la covarianza")

  def procesosCondicionadosGP(self):
    '''
    Genera realizaciones del proceso gaussiano dada la descomposicion cholesky de la matriz de covarianzas y la media correspondiente
    '''
    try:
      return self.Media_pred + np.matmul(self.L_pred,(np.randn(len(self.xtest))).T)
    except:
      raise NameError("El GP no ha sido condicionado a las observaciones")

  def sigmaCalc(self,cov):
    '''
    Desviacion estándar punto a punto del proceso gausiano dado por la matriz de covarianza 
    Es la raiz cuadrado de los elementos de la diagonal de la matriz de covarianzas del proceso generado
    '''
    return np.sqrt(np.diag(cov))


  def correlacionPuntoPunto(self):
    '''
    Obtiene la correlacion punto a punto dado un inputo concreto por el indice i_x
    '''
    try:
      self.corrPP_pred = np.array([ self.K_pred[i,:] for i in range(len(self.K_pred)) ])
    except:
      print(" Proceso gaussiano no entrenado aún, no existe K_pred ")
    self.corrPP_prior = np.array([ self.K_prior[i,:] for i in range(len(self.K_prior)) ])

  
  def derivada_LO_CV(self,*args):
    '''
    Dadas unas observaciones particulares estima la derivada de la suma de los logaritmos de la evaluación de la distribución de probabilidad marginal de verosimilitud a que subconjuntos
    de las observaciones están condicionadas al resto, respecto a los hiperparámetros del modelo. Si se encuentra el máximo de este valor, estamos asegurándonos de que con los hiperparámetros ajustados
    las "predicciones" tras generar trayectorias del proceso gausiano que corresponderían a los subconjuntos excluidos sucesivamente son las más probables que el modelo puede hacer. Al ser la suma, esta optimización 
    es global y abarca la aproximación de observacion/test de todos los subconjuntos.
    '''

    if not isinstance(args[0], np.ndarray):
      raise NameError("Las observaciones dadas para calcular la derivada del logaritmo de la distribución marginal de verosimilitud no es un array con datos")
    else:
      observaciones = args[0]

    # Vector auxiliar alpha y matriz auxiliar Z y la diagonal de la inversa de la distribucion a priori
    try:
      alpha = np.matmul(self.invK,observaciones)
      invK_diag = np.diag(self.invK)
      zeta = [np.matmul(self.invK,self.Derivadas[i]) for i in range(len(self.Derivadas)) ] # Es una lista de matrices
      zeta_K = [np.diag(np.matmul(zeta[i],self.invK)) for i in range(len(self.Derivadas)) ]
      alpha_invK = np.multiply(alpha,1./invK_diag)
    except:
      raise NameError("Necesitas calcular la matriz inversa de covarianzas de la distribución a priori")
    
    termino_1 = [np.matmul(np.matmul(alpha_invK,zeta[i]),alpha) for i in range(len(self.Derivadas)) ] # Debido a Z esto es una lista
    termino_2 = [np.sum(np.multiply(np.multiply(0.5*(1. + np.multiply(np.power(alpha,2),1./invK_diag)),zeta_K[i]),1./invK_diag)) for i in range(len(self.Derivadas)) ] # Debido a zeta_K esto es una lista

    # Calculo. Es el valor de la derivada del logaritmo
    self.derVerLOO = np.asarray([ termino_1[i] + termino_2[i] for i in range(len(self.Derivadas))])
    

In [ ]:


class datosVisualizacion():
  '''
  Esta clase contiene los datos de entrenamiento y realizaciones de los procesos gaussianos para la visualización de los mismos
  '''
  def __init__(self,Titulo):
    self.x = []
    self.y = []
    self.sigma = []
    self.realizacion_dict = {}
    self.titulo = Titulo
    self.inicializacionPlot()

  def inicializacionPlot(self):
    '''
    Inicializa el objeto matplotlib
    '''
    self.fig,self.ax = plt.subplots(nrows=1,ncols=1,figsize=(10,10))
    self.ax.set_title(self.titulo)
    self.fig_hm, self.ax_hm = plt.subplots(nrows=1,ncols=1,figsize=(10,10))

  def add_covariance(self, cov):
    '''
    Añada matriz de covarianza y alternativamente calcula la covarianza del proceso gaussiano sobre el dominio
    '''

    self.cov_GP = cov

  def add_mean(self,X,Mean):
    '''
    Almacena la desviación estándar de un proceso gausiano dado
    '''
    self.x_mean = X
    self.mean = Mean


  def add_sigma(self,X,Sigma):
    '''
    Almacena la desviación estándar de un proceso gausiano dado
    '''
    self.x_sigma = X
    self.sigma = Sigma
    self.meanMarker = 'r-'

  def add_train_data(self,X,Y):
    self.xtrain = X
    self.ytrain = Y
    self.marker = 'b*'

  def add_data(self,X,Y,label):
    self.x.append(X)
    self.y.append(Y)
    self.realizacion_dict[label] = len(self.x)-1

  def add_corrPuntoPunto(self,corrPP, corrInputs):
    '''
    Añade correlaciones punto a punto sobre un dominio dado por los valores corrInputs
    '''
    self.corrPP = corrPP
    self.corr_in = corrInputs

  def add_data_plot(self,label):
    try:
      self.ax.plot(self.x[self.realizacion_dict[label]],self.y[self.realizacion_dict[label]],'g--')
    except:
      print('No existe datos referentes a etiqueta :',label)

  def add_train_data_plot(self):
    try:
      self.ax.plot(self.xtrain,self.ytrain,self.marker)
    except:
      print('No existen datos de entrenamiento')

  def add_mean_plot(self):
    try:
      self.ax.plot(self.x_mean,self.mean,self.meanMarker)
    except:
      print('No existen datos de entrenamiento')



  def contorno_sigma(self):
    '''
    Muestra en pantalla contorno de 1 y 2 veces los valores de sigma del proceso gausiano
    '''
    self.ax.fill(np.concatenate((self.x_sigma,self.x_sigma[::-1])),
             np.concatenate((self.mean - 1.9600 * self.sigma,
                             (self.mean + 1.9600 * self.sigma)[::-1])),
             alpha=.45, fc='y', ec='None', label='95% confidence interval')
    self.ax.fill(np.concatenate((self.x_sigma,self.x_sigma[::-1])),
             np.concatenate((self.mean - 1.000 * self.sigma,
                             (self.mean + 1.000 * self.sigma)[::-1])),
             alpha=.35, fc='b', ec='None', label='68% confidence interval')
    
  def plotCorrPuntoPunto(self,indices):
    '''
    Visualiza correlaciones punto a punto para los inputs dados por sus indices en la lista indices como argumento del metodo
    '''

    ' Generamos y visualizamos las imagen directamente en este metodo'
    self.fig_cpp, self.ax_cpp = plt.subplots(nrows=1,ncols=1,figsize=(10,10))
    try:
      [self.ax_cpp.plot(self.corr_in,self.corrPP[indice],'-') for indice in indices] 
    except:
      print('No existe datos referentes referentes a correlaciones punto a punto')
    self.ax_cpp.grid(True)
    self.fig_cpp.show()



  def mostrarPlot(self):
    '''
    Muestra plot
    '''
    self.add_mean_plot()
    self.add_train_data_plot()
    self.contorno_sigma()
    self.ax.grid(True)
    self.fig.show()

  def hmapCov(self):
    '''
    Visualiza covarianza como campo bidimensional
    '''
    plotRange = np.arange(len(self.cov_GP))
    heatmap = self.ax_hm.pcolormesh(self.cov_GP,cmap='RdBu_r')
    self.ax_hm.axis([plotRange.min(), plotRange.max(), plotRange.max(), plotRange.min()])
    self.fig_hm.colorbar(heatmap, ax=self.ax_hm)
    self.fig_hm.show()




In [ ]:
import time


start_time = time.time()

#x_train = np.linspace(0.1,9.7,5)
#y_train = 1.1*np.sin(2.*x_train) + np.asarray([random.gauss(0.,1.) for i in range(len(x_train))])
x_train = np.array([-7.27980659640218,-6.3565864147485,-6.33349783214072,-5.85186882395103,-4.74692152231896,-4.08519569483122,-3.73024545572045,-2.78487668464404,-2.15092471829579,-0.902510956795564,0.496360463886639,0.756812013112905,1.00714394820637,2.37248404949064,2.46618962012525,4.24396641874347,4.33629137438754,4.89477951284853,5.81238911017895,6.14524950944106])
y_train = np.array([-1.49888963305016,0.569441996545526,0.623268356304418,0.9063943036413,-0.376423208431739,-0.871911241143502,-0.803958546300539,1.07173322993408,2.29393880644365,2.62802707180373,0.412113574676584,-0.297490218195917,-0.660456836687934,-2.66828227995347,-2.09868165955797,-0.965605061863299,-0.758001691987733,-1.32241178751454,-0.725175367478586,-0.449178680954562])

grp = 4
#sigmas = np.linspace(0.01,3.,grp)
#ls = np.linspace(0.01,10.,grp)

sigmas = np.linspace(np.log(0.01),np.log(3.),grp)
ls = np.linspace(np.log(0.01),np.log(10.),grp)

xx, yy = np.meshgrid(ls,sigmas)

sigmas = np.exp(sigmas)
ls = np.exp(ls)


probabilidades = np.zeros((grp,grp))

s = 1.

gp_prueba = gaussProcess(x_train,y_train,[0.,0.,0.])
gp_prueba.compCov(lambda x,y,sh,lh,sigmah : gp_prueba.Kernels.exponencialCuadrada(x,y,sh,lh)+gp_prueba.Kernels.ruidoBlanco(x,y,sigmah)) # Aqui especificamos los hiperparametros
funciones = np.asarray([lambda x,y,sh,lh,sigmah: gp_prueba.Kernels.expCua_der_s(x,y,sh,lh) , lambda x,y,sh,lh,sigmah: gp_prueba.Kernels.expCua_der_l(x,y,sh,lh), lambda x,y,sh,lh,sigmah: gp_prueba.Kernels.rB_der_sigma(x,y,sigmah)])
gp_prueba.compDerCov(funciones)


i = 0
for sigma in sigmas:

  j = 0
  for l in ls:
    gp_prueba.Hiper = [s,l,sigma]
    gp_prueba.log_prob_verosimilitud_datos(*gp_prueba.Hiper)
    probabilidades[i,j] = gp_prueba.log_marg_y
    resultado = gp_prueba.der_log_prob_ver_hiper(*gp_prueba.Hiper)
    #print("logaritmo de la probabilidad de verosimilitud e hiperparametros:",gp_prueba.log_marg_y)
  
    j = j + 1
  
  i = i + 1


fig_3, ax_3 = plt.subplots()

plotRange_sig = sigmas
plotRange_l = ls

heatmap = ax_3.pcolormesh(xx,yy,probabilidades,cmap='RdBu_r',vmin = probabilidades.min(), vmax = probabilidades.max())
#ax_3.axis([plotRange_l.min(), plotRange_l.max(), plotRange_sig.min(), plotRange_sig.max()])
fig_3.colorbar(heatmap, ax=ax_3)

#ax_3.set_xscale('log')
#ax_3.set_yscale('log')

fig_3.show()

#im = ax_3.imshow(probabilidades)

#fig_3.colorbar(im, ax=ax_3)

#fig_3.show()

print("--- %s seconds ---" % (time.time() - start_time))

#print("partida :",[1.,1.,0.3])
ppp = pruebaHMC()
#afdaf = [np.random.randn(),np.random.randn(),np.random.randn(),np.random.randn(),np.random.randn()]
#afdaf = [10485.50142632,   9058.50096209 , -8420.49975872, -10909.50244681 ,-311.49708436]
#print("partida :",afdaf)

busqueda = hmc_annealing([1.,1.,0.3],400.,.4,10,gp_prueba)
#busqueda = hmc_annealing(afdaf,400.,0.4,10,ppp)
busqueda.annealing()
print("Finalmente el valor minimo calculado :",gp_prueba.log_prob_verosimilitud_datos(busqueda.ObjetoHMC.Xo), " teorico :", [1.,1.1,0.15])

--- 0.3672933578491211 seconds ---
Paso de enfriamiento dado, temp actual : 400.0  aceptado promedio : 0.9943473137269142  dt : 5.891792449153029  n : 10  total pasos probados : 1006
Paso de enfriamiento dado, temp actual : 296.32728827268716  aceptado promedio : 0.983865264038652  dt : 89.38653702953387  n : 10  total pasos probados : 1019
Paso de enfriamiento dado, temp actual : 219.52465443761056  aceptado promedio : 0.9801362335233741  dt : 1356.1158291108536  n : 10  total pasos probados : 1016
Paso de enfriamiento dado, temp actual : 162.62786389623966  aceptado promedio : 0.9943310719724084  dt : 19974.882505330555  n : 10  total pasos probados : 1005
Paso de enfriamiento dado, temp actual : 120.47768476488085  aceptado promedio : 0.994255046145116  dt : 294219.6547940638  n : 10  total pasos probados : 1001
Paso de enfriamiento dado, temp actual : 89.25206405937193  aceptado promedio : 0.9773328217203899  dt : 4463713.936808294  n : 10  total pasos probados : 1022
Paso de enfri

In [52]:
class hmc_annealing():

  def __init__(self,x,temp,dt,n,gp):
    self.Temp = temp
    self.Dt = dt
    self.N = n
    self.GP = gp
    self.ObjetoHMC = self.hybridMontecarlo(x,self.Temp,self.Dt,self.N,self.GP)

  def pasoAnnealing(self):
    #print("Fuerzas involucradas iniciales :",self.GP.der_log_prob_ver_hiper(*self.ObjetoHMC.Xo))
    self.ObjetoHMC.Po = np.sqrt(self.Temp)*np.random.randn(len(self.ObjetoHMC.Xo)) # Siempre muestreamos la distribucion de momentos antes de cada trayectoria
    #self.ObjetoHMC.Po = self.Temp*np.random.randn(len(self.ObjetoHMC.Xo)) # Siempre muestreamos la distribucion de momentos antes de cada trayectoria

    self.Ho = self.GP.log_prob_verosimilitud_datos(*self.ObjetoHMC.Xo) + 0.5*np.sum(np.power(self.ObjetoHMC.Po,2),axis=0)
    self.ObjetoHMC.ejecutarTrayectoria(self.Dt)
    #if (self.ObjetoHMC.Xo < 0.).any():
    #  # Hiperparametros deben ser positivos
    #  self.acc_prob = 0.
    #else:
    self.H = self.GP.log_prob_verosimilitud_datos(*self.ObjetoHMC.Xo) + 0.5*np.sum(np.power(self.ObjetoHMC.Po,2),axis=0)
    #print("tras trayectoria :",self.ObjetoHMC.Xo)
    self.Dif_H = self.H-self.Ho
    #print("Dif_H :",self.Dif_H)
    self.acc_prob = np.minimum(1.,np.exp(-(1./self.Temp)*self.Dif_H))

  def aceptacion(self):
    unif = np.random.uniform()
    if unif > self.acc_prob:
      # No se acepta la nueva configuracon
      self.ObjetoHMC.Xo = self.prev_Xo
    else:
      # Se acepta
      self.aceptados = self.aceptados + 1

    self.Total = self.Total + 1
    self.acc_prob_a = self.acc_prob_a*((self.Total-1)/self.Total) + (self.acc_prob/self.Total)
    self.SubTotal = self.SubTotal + 1
    if (self.SubTotal > 10):
      if (self.acc_prob_a < 0.45):
        # Debemos intentar mejorar la aceptacion
        self.Dt = self.Dt - self.Dt*(0.03)
      elif (self.acc_prob_a > 0.55):
        # Debemos intentar reducir la aceptacióón
        self.Dt = self.Dt + self.Dt*(0.03)
      self.SubTotal = 0
    #print(unif,self.acc_prob,self.aceptados)

  def reducirT(self,To,astep):
    self.Temp = To*np.exp(-0.3*float(astep))

  def annealing(self):
    an_step = 0
    self.To = self.Temp
    while an_step < 50:
      self.aceptados = 0
      self.Total = 0
      self.acc_prob_a = 0.
      self.SubTotal = 0.
      while self.aceptados < 1000:
        self.prev_Xo = self.ObjetoHMC.Xo
        self.pasoAnnealing()
        self.aceptacion()
      an_step = an_step + 1
      print("Paso de enfriamiento dado, temp actual :",self.Temp, " aceptado promedio :", self.acc_prob_a," dt :", self.Dt, " n :",self.N, " total pasos probados :", self.Total)
      self.reducirT(self.To,an_step)
    print("Resultado final:",self.ObjetoHMC.Xo)
      

  class hybridMontecarlo():

    def __init__(self,x,temp,dt,n,gp):
      self.Xo = np.asarray(x) # En este formalismo de procesos gausianos son una lista de hiperparámetros
      self.Po = None
      self.Temp = temp
      self.Dt = dt
      self.t = 0. # Inicializacion del tiempo
      self.N = n
      self.Nd = len(self.Xo)
      self.GP = gp 
      #self.trayectoria, self.ax_tray = plt.subplots(1,1)
    def updateStepIter(self,):
      '''
      Ejecuta avance en el integrador simpléctico y actualiza las fuerzas calculadas
      '''
      self.update_x_verlet()
      self.F = self.GP.der_log_prob_ver_hiper(*self.X)
      self.update_p_verlet()
      self.Fo = self.F
      self.t += self.Dt

    def update_x_verlet(self):
      '''
      La operacion se realiza sobre ndarray
      '''
      self.X = self.Xo + self.Dt*(self.Po + (0.5*self.Dt*self.Fo))
      self.Xo = self.X

    def update_p_verlet(self):
      '''
      La operacion se realiza sobre ndarray
      '''
      self.P = self.Po + 0.5*self.Dt*(self.Fo + self.F)
      self.Po = self.P

    #@numba.njit
    def ejecutarTrayectoria(self,tray_dt):
      '''
      Desarrolla una trayectoria en el integrador simplectico de self.N iteraciones
      '''
      self.Dt = tray_dt
      self.Fo = self.GP.der_log_prob_ver_hiper(*self.Xo)   
      for index_t in range(self.N):
        self.updateStepIter()
        #self.ploteado(False)
      #self.ploteado(True)


    def ploteado(self,mostrar):
      '''
      Muestra punto x integrado hasta ahora frente al indice del tiempo
      '''
      for i in range(self.Nd):
        self.ax_tray.plot(self.t,self.X[i],'ob')
      if mostrar == True:
        print("se ha mostrado")
        self.trayectoria.show()

In [11]:
class pruebaHMC():
  def __init__(self):
    pass
  
  def log_prob_verosimilitud_datos(self,*hiper):
    arrayEntrada = np.asarray(hiper)
    return (1./10.)*np.sum(np.sin(8*np.pi*arrayEntrada)/np.sin(2*np.pi*arrayEntrada))

  def der_log_prob_ver_hiper(self,*hiper):
    return np.asarray([-(np.pi/5.)*(   (4*np.cos(8*np.pi*i)*np.sin(2*np.pi*i)-np.sin(8*np.pi*i)*np.cos(2*np.pi*i)     ) /  np.power(np.sin(2*np.pi*i),2)   ) for i in hiper])
